In [1]:
import rmgpy.chemkin
import os 
import re
import numpy as np
import cantera as ct
import pandas as pd

In [3]:
full_path = '/work/westgroup/nora/Code/'

In [7]:
#preliminary model 
pre_RMG_chem = full_path + 'Advanced_Kinetics_Class/project/models/RMG/Fluorine_lib/chemkin/copies/copy_chem_annotated_208.inp'
pre_RMG_dict = full_path + 'Advanced_Kinetics_Class/project/models/RMG/Fluorine_lib/chemkin/species_dictionary.txt'
pre_RMG_trans = full_path + 'Advanced_Kinetics_Class/project/models/RMG/Fluorine_lib/chemkin/tran.dat'

pre_species_list, pre_reaction_list = rmgpy.chemkin.load_chemkin_file(pre_RMG_chem, dictionary_path=pre_RMG_dict, transport_path=pre_RMG_trans)

#current model

curr_RMG_chem = full_path + 'projects/PFAS/ESSCI/models/RMG/4FP/RMG/chemkin/copies/copy_chem_annotated.inp'
curr_RMG_dict = full_path + 'projects/PFAS/ESSCI/models/RMG/4FP/RMG/chemkin/species_dictionary.txt'
curr_RMG_trans = full_path + 'projects/PFAS/ESSCI/models/RMG/4FP/RMG/chemkin/tran.dat'

curr_species_list, curr_reaction_list = rmgpy.chemkin.load_chemkin_file(curr_RMG_chem, dictionary_path=curr_RMG_dict, transport_path=curr_RMG_trans)

In [148]:
#first find all the estimated F and H abstraction reactions in the preliminary RMG model 

pre_H_Abstr_list = []
pre_F_Abstr_list = []

#preliminary model 
for RMG_rxn in pre_reaction_list:
    try: 
        if RMG_rxn.family=='F_Abstraction':
            reactant_indices = sorted([spec.index for spec in RMG_rxn.reactants])
            product_indices = sorted([spec.index for spec in RMG_rxn.products])
            append_this = [RMG_rxn, reactant_indices, product_indices] 
            pre_F_Abstr_list.append(append_this)
        if RMG_rxn.family=='H_Abstraction':
            reactant_indices = sorted([spec.index for spec in RMG_rxn.reactants])
            product_indices = sorted([spec.index for spec in RMG_rxn.products])
            append_this = [RMG_rxn, reactant_indices, product_indices] 
            pre_H_Abstr_list.append(append_this)
    except AttributeError as e:
        continue

In [149]:
len(pre_H_Abstr_list) #should be 382

382

In [133]:
len(pre_F_Abstr_list) #should be 133, great

133

In [108]:
len(top_sens_rxns) #total reactions is 3904, so 3903 is fine since there is a rxn that used N2 

3903

In [135]:
#now go through the sensitivity csv file and see if any of these match preliminary sens rxns  
top_sens_rxns = []

file_path = './preliminary/sensitivities.csv'
df = pd.read_csv(file_path)
for index, row in df.iterrows():
    try: 
        #get the info
        cantera_index = row['k_s']
        sens = row['sensitivity']
        cantera_eq = row['cantera equation']
        prods = eval(row['cantera products'])
        reacts = eval(row['cantera reactants'])
        
        prod_indices = sorted([int(re.search('\(([0-9]+)\)', x).group(1)) for x in prods.keys()])
        react_indices = sorted([int(re.search('\(([0-9]+)\)', x).group(1)) for x in reacts.keys()])

        print(index, react_indices, prod_indices)
        top_sens_rxns.append([index, react_indices, prod_indices])
    except AttributeError as e: 
        continue
    

0 [3, 5] [2, 6]
1 [2, 12] [5, 13]
2 [3, 45] [6, 48]
3 [3, 1707] [6, 1963]
4 [5, 49] [38, 51]
5 [1, 6] [45, 79]
6 [2, 45] [38, 49]
7 [1, 6] [17, 77]
8 [3, 147] [6, 287]
9 [2, 43] [5, 49]
10 [1, 5] [86]
11 [1, 2] [4, 84]
12 [79] [12, 46]
13 [1, 5] [45, 56]
14 [45, 86] [44, 237]
15 [14] [5, 12]
16 [1, 37] [38, 84]
17 [3, 46] [2, 47]
18 [6, 56] [17, 47]
19 [2, 84] [1, 6]
20 [3, 73] [6, 621]
21 [5, 27] [12, 18]
22 [2, 46] [4, 40]
23 [12, 37] [51]
24 [4, 37] [2, 38]
25 [45, 58] [540]
26 [3, 43] [6, 49]
27 [46, 86] [41, 237]
28 [45, 147] [44, 150]
29 [45, 79] [27, 44]
30 [45, 58] [147]
31 [2, 12] [5, 13]
32 [2, 14] [4, 12]
33 [2, 1707] [6, 58]
34 [45, 540] [44, 150]
35 [6, 56] [14, 46]
36 [45, 58] [84]
37 [45, 56] [5, 141]
38 [3, 17] [5, 640]
39 [3, 82] [2, 49]
40 [45, 58] [37, 83]
41 [79, 82] [27, 42]
42 [45, 84] [44, 83]
43 [43, 57] [1]
44 [13, 38] [2, 51]
45 [10, 43] [2, 49]
46 [3, 1963] [6, 11019]
47 [12, 17] [27]
48 [6, 18] [5, 19]
49 [79, 210] [27, 83]
50 [3, 84] [6, 85]
51 [2, 57] [4, 

H Abstraction 

In [121]:
count = 0
for [index, sens_react_indices, sens_prod_indices] in top_sens_rxns: 
    for [RMG_rxn, H_reactant_indices, H_product_indices] in pre_H_Abstr_list:
        #check forward
        if sens_react_indices == H_reactant_indices and sens_prod_indices == H_product_indices:
            print(f'found forward match for {index}. {sens_react_indices}, {sens_prod_indices} ')
            count+=1
            print(RMG_rxn.kinetics.get_rate_coefficient(1400, 101325))
        #check reverse
        if sens_react_indices == H_product_indices and sens_prod_indices == H_reactant_indices:
            print(f'found reverse match for {index}')
            count+=1
        
print(count)
     
#it found a 391 matches. This means some were duplicates 

found forward match for 11. [1, 2], [4, 84] 
1252040.2078279117
found forward match for 16. [1, 37], [38, 84] 
51576718.56458096
found forward match for 19. [2, 84], [1, 6] 
431759.8813111978
found forward match for 24. [4, 37], [2, 38] 
51931107.91617032
found forward match for 33. [2, 1707], [6, 58] 
3519762346.7007513
5


F Abstraction 

In [136]:
count = 0
for [index, sens_react_indices, sens_prod_indices] in top_sens_rxns: 
    for [RMG_rxn, F_reactant_indices, F_product_indices] in pre_F_Abstr_list:
        #check forward
        if sens_react_indices == F_reactant_indices and sens_prod_indices == F_product_indices:
            print(f'found forward match for {index}. {sens_react_indices}, {sens_prod_indices} ')
            count+=1
            print(RMG_rxn.kinetics.get_rate_coefficient(1400, 101325))
        #check reverse
        if sens_react_indices ==F_product_indices and sens_prod_indices == F_reactant_indices:
            print(f'found reverse match for {index}')
            count+=1
        
print(count)
     
#it found a 135 matches. This means some were duplicates 

found forward match for 4. [5, 49], [38, 51] 
219.45564479495954
found forward match for 60. [5, 44], [38, 45] 
31.31946430188549
found forward match for 68. [5, 83], [38, 210] 
281.2739920575104
found forward match for 110. [5, 150], [38, 210] 
3303.8778317035676
found forward match for 203. [5, 58], [21, 38] 
108.22346074908282
found forward match for 206. [5, 150], [38, 210] 
3303.8778317035676
found forward match for 368. [1, 5], [38, 125] 
293.77722772573185
found forward match for 459. [5, 83], [38, 210] 
281.2739920575104
found forward match for 506. [5, 46], [17, 38] 
141.3104858285861
found forward match for 613. [5, 56], [28, 38] 
111.38064175558844
found forward match for 644. [42, 51], [49, 82] 
4.631031390216474
found forward match for 792. [1, 5], [38, 125] 
293.77722772573185
found forward match for 831. [5, 55], [38, 73] 
205.7210728539559
found forward match for 843. [42, 45], [44, 82] 
4.319103401635913
found forward match for 861. [5, 55], [38, 72] 
170.8883310666782

In [139]:
for rxn in curr_reaction_list:
    if 'H(5) + 2,3,3,3-tetrafluoropropene(1) <=> HF(38) + C=C(F)[C](F)F(125)' in str(rxn):
        print(rxn.kinetics.get_rate_coefficient(1400, 101325))

271.14836920463387
120.03225153992007


In [129]:
for rxn in curr_reaction_list:
    
    print(rxn)

O2(3) + H(5) <=> O(6) + OH(2)
O(6) + H2(7) <=> H(5) + OH(2)
OH(2) + H2(7) <=> H(5) + H2O(4)
OH(2) + OH(2) <=> O(6) + H2O(4)
H2(7) <=> H(5) + H(5)
Ar(8) + H2(7) <=> Ar(8) + H(5) + H(5)
He(9) + H2(7) <=> He(9) + H(5) + H(5)
O(6) + O(6) <=> O2(3)
Ar(8) + O(6) + O(6) <=> Ar(8) + O2(3)
He(9) + O(6) + O(6) <=> He(9) + O2(3)
O(6) + H(5) <=> OH(2)
H2O(4) <=> H(5) + OH(2)
H2O(4) + H2O(4) <=> H(5) + OH(2) + H2O(4)
O2(3) + H(5) <=> HO2(10)
H(5) + HO2(10) <=> O2(3) + H2(7)
H(5) + HO2(10) <=> OH(2) + OH(2)
H(5) + HO2(10) <=> O(6) + H2O(4)
O(6) + HO2(10) <=> O2(3) + OH(2)
OH(2) + HO2(10) <=> O2(3) + H2O(4)
HO2(10) + HO2(10) <=> O2(3) + H2O2(11)
H2O2(11) <=> OH(2) + OH(2)
H(5) + H2O2(11) <=> OH(2) + H2O(4)
H(5) + H2O2(11) <=> HO2(10) + H2(7)
O(6) + H2O2(11) <=> OH(2) + HO2(10)
OH(2) + H2O2(11) <=> HO2(10) + H2O(4)
O(6) + CO(12) <=> CO2(13)
O2(3) + CO(12) <=> O(6) + CO2(13)
OH(2) + CO(12) <=> H(5) + CO2(13)
HO2(10) + CO(12) <=> OH(2) + CO2(13)
HCO(14) <=> H(5) + CO(12)
H(5) + HCO(14) <=> H2(7) + CO(12

In [150]:
#now lets look at the sens of the curr model and see if theres anything there
#first find all the estimated F and H abstraction reactions in the preliminary RMG model 

curr_H_Abstr_list = []
curr_F_Abstr_list = []

#preliminary model 
for RMG_rxn in curr_reaction_list:
    try: 
        if RMG_rxn.family=='F_Abstraction':
            reactant_indices = sorted([spec.index for spec in RMG_rxn.reactants])
            product_indices = sorted([spec.index for spec in RMG_rxn.products])
            append_this = [RMG_rxn, reactant_indices, product_indices] 
            curr_F_Abstr_list.append(append_this)
        if RMG_rxn.family=='H_Abstraction':
            reactant_indices = sorted([spec.index for spec in RMG_rxn.reactants])
            product_indices = sorted([spec.index for spec in RMG_rxn.products])
            append_this = [RMG_rxn, reactant_indices, product_indices] 
            curr_H_Abstr_list.append(append_this)
    except AttributeError as e:
        continue
        

In [151]:
len(curr_F_Abstr_list)

36

In [152]:
#now go through the sensitivity csv file and see if any of these match preliminary sens rxns  
top_sens_rxns = []

file_path = './new/sensitivities.csv'
df = pd.read_csv(file_path)
for index, row in df.iterrows():
    try: 
        #get the info
        cantera_index = row['k_s']
        sens = row['sensitivity']
        cantera_eq = row['cantera equation']
        prods = eval(row['cantera products'])
        reacts = eval(row['cantera reactants'])
        
        prod_indices = sorted([int(re.search('\(([0-9]+)\)', x).group(1)) for x in prods.keys()])
        react_indices = sorted([int(re.search('\(([0-9]+)\)', x).group(1)) for x in reacts.keys()])

        print(index, react_indices, prod_indices)
        top_sens_rxns.append([index, react_indices, prod_indices])
    except AttributeError as e: 
        continue
    

0 [3, 45] [6, 48]
1 [3, 5] [2, 6]
2 [3, 147] [6, 287]
3 [2, 12] [5, 13]
4 [1, 6] [17, 77]
5 [3, 46] [2, 47]
6 [1, 6] [45, 79]
7 [56] [22, 38]
8 [27, 45] [12, 153]
9 [38, 40] [41]
10 [1, 5] [86]
11 [2, 84] [1, 6]
12 [45, 58] [147]
13 [147] [84]
14 [45, 58] [84]
15 [1, 2] [4, 84]
16 [38, 49] [2, 45]
17 [45, 86] [44, 237]
18 [45, 1811] [287]
19 [1, 2] [45, 3443]
20 [3, 5] [10]
21 [12, 54] [287]
22 [79] [12, 46]
23 [5, 58] [21, 38]
24 [46, 86] [41, 237]
25 [1, 2] [595]
26 [45, 147] [44, 150]
27 [2, 27] [4, 20]
28 [147] [540]
29 [1, 5] [45, 56]
30 [5, 79] [27, 38]
31 [45, 58] [540]
32 [3, 540] [49, 1816]
33 [2, 12] [5, 13]
34 [10, 43] [2, 49]
35 [45, 540] [44, 150]
36 [3, 54] [944]
37 [46, 147] [41, 150]
38 [5, 44] [38, 45]
39 [45, 3443] [1581]
40 [45, 84] [44, 83]
41 [4, 37] [2, 38]
42 [45, 79] [27, 44]
43 [540] [84]
44 [3, 84] [6, 85]
45 [14] [5, 12]
46 [46, 540] [41, 150]
47 [45, 54] [44, 55]
48 [46, 84] [41, 83]
49 [6, 18] [5, 19]
50 [1, 2] [1581]
51 [57] [38, 58]
52 [6459] [49, 1816]
5

In [153]:
count = 0
for [index, sens_react_indices, sens_prod_indices] in top_sens_rxns: 
    for [RMG_rxn, F_reactant_indices, F_product_indices] in curr_F_Abstr_list:
        #check forward
        if sens_react_indices == F_reactant_indices and sens_prod_indices == F_product_indices:
            print(f'found forward match for {index}. {sens_react_indices}, {sens_prod_indices} ')
            count+=1
            print(RMG_rxn.kinetics.get_rate_coefficient(1400, 101325))
        #check reverse
        if sens_react_indices ==F_product_indices and sens_prod_indices == F_reactant_indices:
            print(f'found reverse match for {index}')
            count+=1
        
print(count)
#found 36, exactly like it should have 

found forward match for 23. [5, 58], [21, 38] 
96.61008477864408
found forward match for 38. [5, 44], [38, 45] 
32.131119240212556
found forward match for 98. [5, 56], [28, 38] 
99.46421633432925
found forward match for 101. [5, 41], [38, 46] 
62.47754680418254
found forward match for 290. [5, 46], [17, 38] 
85.69624334404554
found forward match for 404. [1, 5], [38, 125] 
271.14836920463387
found forward match for 510. [5, 47], [14, 38] 
252.73038442417868
found forward match for 545. [1, 5], [38, 125] 
271.14836920463387
found forward match for 649. [45, 56], [28, 44] 
1.4341368248778554
found forward match for 651. [21, 41], [46, 58] 
0.7687039642871061
found forward match for 663. [45, 58], [21, 44] 
0.3521508827694155
found forward match for 736. [5, 1811], [20, 38] 
4500.937789799995
found forward match for 767. [41, 45], [44, 46] 
0.7156817322750508
found forward match for 799. [1, 45], [44, 125] 
1.3960536005624689
found forward match for 817. [40, 45], [16, 44] 
2.155977048715

In [160]:
for rxn in pre_reaction_list:
    if "H(5) + CH2F(46) <=> HF(38) + CH2(T)(17)" in str(rxn):
        print(rxn.kinetics.get_rate_coefficient(1400, 101325))

141.3104858285861


In [155]:
for rxn in pre_reaction_list:
        print(rxn)

O2(3) + H(5) <=> O(6) + OH(2)
O(6) + H2(7) <=> H(5) + OH(2)
OH(2) + H2(7) <=> H(5) + H2O(4)
OH(2) + OH(2) <=> O(6) + H2O(4)
H2(7) <=> H(5) + H(5)
Ar(8) + H2(7) <=> Ar(8) + H(5) + H(5)
He(9) + H2(7) <=> He(9) + H(5) + H(5)
O(6) + O(6) <=> O2(3)
Ar(8) + O(6) + O(6) <=> Ar(8) + O2(3)
He(9) + O(6) + O(6) <=> He(9) + O2(3)
O(6) + H(5) <=> OH(2)
H2O(4) <=> H(5) + OH(2)
H2O(4) + H2O(4) <=> H(5) + OH(2) + H2O(4)
H(5) + HO2(10) <=> O2(3) + H2(7)
H(5) + HO2(10) <=> OH(2) + OH(2)
H(5) + HO2(10) <=> O(6) + H2O(4)
O(6) + HO2(10) <=> O2(3) + OH(2)
OH(2) + HO2(10) <=> O2(3) + H2O(4)
HO2(10) + HO2(10) <=> O2(3) + H2O2(11)
H(5) + H2O2(11) <=> OH(2) + H2O(4)
H(5) + H2O2(11) <=> HO2(10) + H2(7)
O(6) + H2O2(11) <=> OH(2) + HO2(10)
OH(2) + H2O2(11) <=> HO2(10) + H2O(4)
O(6) + CO(12) <=> CO2(13)
O2(3) + CO(12) <=> O(6) + CO2(13)
OH(2) + CO(12) <=> H(5) + CO2(13)
HO2(10) + CO(12) <=> OH(2) + CO2(13)
HCO(14) <=> H(5) + CO(12)
H(5) + HCO(14) <=> H2(7) + CO(12)
O(6) + HCO(14) <=> OH(2) + CO(12)
O(6) + HCO(14) <